<a href="https://colab.research.google.com/github/mahfuz640/car_number_plate_detect/blob/main/car_number_plate_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
#!/bin/bash
!kaggle datasets download sifatkhan69/bangladeshi-vehicle-license-plate

Dataset URL: https://www.kaggle.com/datasets/sifatkhan69/bangladeshi-vehicle-license-plate
License(s): MIT
 90% 462M/514M [00:02<00:00, 90.7MB/s]
100% 514M/514M [00:02<00:00, 182MB/s] 


In [6]:
import os
import shutil
import random
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import kagglehub
from sklearn.model_selection import train_test_split

# Install required packages
!pip install ultralytics

from ultralytics import YOLO
import torch

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("GPU Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
GPU Available: True
Device: Tesla T4


In [12]:
root_path = "/kaggle/input/bangladeshi-vehicle-license-plate"

print("Inspecting contents of:", root_path)
for root, dirs, files in os.walk(root_path):
    level = root.replace(root_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:  # show max 5 files per folder
        print(f"{subindent}{f}")

Inspecting contents of: /kaggle/input/bangladeshi-vehicle-license-plate


In [13]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# Paths
src_dir = "/content/bangladeshi-vehicle-license-plate/Bangladeshi_Vehicle_Image_with_visible_license_plate"
project_dir = "/kaggle/working/yolo_license_plate"
dataset_dir = f"{project_dir}/datasets/license_plate"

# Create required YOLO directories
folders = [
    "images/train", "images/val", "images/test",
    "labels/train", "labels/val", "labels/test"
]
for folder in folders:
    os.makedirs(os.path.join(dataset_dir, folder), exist_ok=True)

# List all images
image_files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_files.sort()

# Step 1: Split into 70% train, 20% test
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Step 2: Split train into 70% train, 30% val
train_files, val_files = train_test_split(train_files, test_size=0.3, random_state=42)

# Dataset dictionary
splits = {
    'train': train_files,
    'val': val_files,
    'test': test_files
}

# Copy files to YOLO directory
for split, files in splits.items():
    for img_file in files:
        # Copy image
        src_img = os.path.join(src_dir, img_file)
        dst_img = os.path.join(dataset_dir, 'images', split, img_file)
        shutil.copy(src_img, dst_img)

        # Copy corresponding label
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_label = os.path.join(src_dir, label_file)
        dst_label = os.path.join(dataset_dir, 'labels', split, label_file)
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)

print("✅ Dataset split (70/30 then 70/30) complete and files copied.")

# Summary
for category in ['images', 'labels']:
    for split in ['train', 'val', 'test']:
        folder = os.path.join(dataset_dir, category, split)
        print(f"{category}/{split}: {len(os.listdir(folder))} files")

✅ Dataset split (70/30 then 70/30) complete and files copied.
images/train: 1080 files
images/val: 463 files
images/test: 386 files
labels/train: 1080 files
labels/val: 463 files
labels/test: 386 files


In [10]:
import zipfile

zip_path = '/content/bangladeshi-vehicle-license-plate.zip'
extract_path = '/content/bangladeshi-vehicle-license-plate'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Successfully extracted {zip_path} to {extract_path}")

Successfully extracted /content/bangladeshi-vehicle-license-plate.zip to /content/bangladeshi-vehicle-license-plate


In [14]:
import yaml

data_yaml = {
    'train': '/kaggle/working/yolo_license_plate/datasets/license_plate/images/train',
    'val': '/kaggle/working/yolo_license_plate/datasets/license_plate/images/val',
    'test': '/kaggle/working/yolo_license_plate/datasets/license_plate/images/test',
    'nc': 1,
    'names': ['license_plate']
}

with open('/kaggle/working/yolo_license_plate/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml created.")

✅ data.yaml created.


In [15]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8s.pt')  # You can also use yolov11m.pt or yolov11x.pt

# Train
model.train(
    data='/kaggle/working/yolo_license_plate/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    project='/kaggle/working/yolo_license_plate',
    name='yolov11_license_plate',
    device=0  # or 'cpu'
)

Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_license_plate/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_license_plate, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plot

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cf046f39820>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [16]:
metrics = model.val(
    data='/kaggle/working/yolo_license_plate/data.yaml',
    split='test'
)
print(metrics)

Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1489.7±1027.0 MB/s, size: 108.1 KB)
val: Scanning /kaggle/working/yolo_license_plate/datasets/license_plate/labels/test... 386 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 386/386 2299.3it/s 0.2s
val: New cache created: /kaggle/working/yolo_license_plate/datasets/license_plate/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 2.7it/s 9.4s
                   all        386        394      0.983      0.982      0.991      0.625
Speed: 0.9ms preprocess, 7.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /kaggle/working/yolo_license_plate/yolov11_license_plate2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.uti

In [17]:
import zipfile
import os

base_dir = '/kaggle/working/yolo_license_plate/yolov11_license_plate'
zip_path = '/kaggle/working/yolov11_license_plate_full_backup.zip'

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for folder, _, files in os.walk(base_dir):
        for file in files:
            full_path = os.path.join(folder, file)
            arcname = os.path.relpath(full_path, base_dir)
            zipf.write(full_path, arcname=arcname)

print("✅ Full model folder zipped as:", zip_path)

✅ Full model folder zipped as: /kaggle/working/yolov11_license_plate_full_backup.zip
